In [22]:
%load_ext autoreload
%autoreload 2

In [16]:
# Run this notebook outside of main module tree
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [33]:
from performance import *

In [34]:
timing = pd.read_csv("../eeg_tests/timing.csv")
eeg_data = "../data/EEG"

In [35]:
perf_df = read_all_user_success(timing, eeg_data, targets = ['two-back', 'three-back'])

ValueError while adjusting timing:  cannot convert float NaN to integer This test will be lost:  three-back From user:  P10
ValueError while adjusting timing:  cannot convert float NaN to integer This test will be lost:  stroop-log From user:  P10


Could not find user:  P15 Error:  could not find that file!


ValueError while adjusting timing:  cannot convert float NaN to integer This test will be lost:  two-back From user:  P17


In [ ]:
perf_df